## In NoteBook

说明:

+ 在NoteBook中运行
+ 主要目标是翻译文章，了解作者思路
+ 顺便了解包改编后，策略是否是否能够正常运行
+ 适用于python3.6
+ 暂时没有涉及策略优化部分

In [1]:
%load_ext zipline

到目前为止，我们已经在研究中创建并运行了一条管道。现在我们转到IDE。首先，我们创建一个骨架算法，导入`Pipeline`，并为其添加一个`make_pipeline`函数，以创建一个空管道。

In [2]:
from zipline.pipeline import Pipeline

def initialize(context):
    my_pipe = make_pipeline()

def make_pipeline():
    return Pipeline()

## Attaching a Pipeline

回想一下，在研究中，我们运行了`make_pipeline`来创建管道对象的实例，并使用`run_pipeline`在指定的日期范围内运行管道。我们不能在算法中安全地做到这一点，我们必须以某种方式允许模拟为我们运行我们的管道。为了让仿真运行我们的管道，我们必须将管道与attach_pipeline连接起来。

`attach_pipeline`函数需要两个参数：对我们的管道对象的引用，以及可以任意决定的管道的字符串名称。让我们导入`attach_pipeline`并将我们的空管道连接到我们的骨架示例中。

In [3]:
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

现在管道已连接，它将模拟每一天运行一次。如果算法在2016年6月6日星期一至2016年6月10日星期五进行回测或实时交易，我们的管道将在每周的每一天运行一次（总共5次）。附加管道将每天产生一个新的数据框输出。研究中的日输出与`run_pipeline`的输出类似。但是，我们算法中的输出数据框不包括索引中的日期，因为当前模拟日期暗示为我们管道计算的日期。

## Pipeline Output

管道的输出可以在`before_trading_start`中使用`pipeline_output`进行检索。`pipeline_output`需要附加管道的名称作为参数，并返回模拟中当前日期的输出数据框。让我们导入`pipeline_output`并修改我们的骨架示例，以便每天在上下文中存储我们的管道输出。

In [4]:
#%%zipline --start 2016-1-1 --end 2018-1-1
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline, pipeline_output

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context.
    #output = pipeline_output('my_pipeline')
    context.output = pipeline_output('my_pipeline')

我们的骨架示例现在每天生成3000行(期间有效证券数量)和默认指标数量列的empy数据框。 输出数据框将如下所示（请注意，该索引不再像研究中的MultiIndex一样）：

## Using Our Pipeline From Research

为了在我们的算法中包含我们在前一课中创建的管道，我们可以简单地将我们在研究中编写的`make_pipeline`函数以及所需的import语句复制到我们的算法中。以下内容将运行我们的管道，并在每天的上下文中存储150行和2列（多和空）的输出数据框。

In [5]:
%%zipline --start 2016-1-1 --end 2018-1-1
from zipline.pipeline import Pipeline
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.builtin import QTradableStocks

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    """
    Create our pipeline.
    """

    # Base universe set to the QTradableStocksUS.
    base_universe = QTradableStocks()

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=10,
        mask=base_universe
    )

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=30,
        mask=base_universe
    )

    percent_difference = (mean_10 - mean_30) / mean_30

    # Filter to select securities to short.
    shorts = percent_difference.top(75)

    # Filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for all securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        },
        screen=(securities_to_trade),
    )

def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context
    output = pipeline_output('my_pipeline')

[2018-04-27 21:39:00.502557] INFO: zipline.finance.metrics.tracker: Simulated 488 trading days
first open: 2016-01-04 01:31:00+00:00
last close: 2017-12-29 07:00:00+00:00


,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2016-01-04 05:35:00+00:00,NaN,0.0,NaN,-0.070206,NaN,NaN,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,1,[],0.0
2016-01-05 07:00:00+00:00,0.0,0.0,0.0,-0.067603,0.819490,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,2,[],0.0
2016-01-06 07:00:00+00:00,0.0,0.0,0.0,-0.051246,0.745908,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,3,[],0.0
2016-01-07 02:00:00+00:00,0.0,0.0,0.0,-0.117026,0.738897,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,4,[],0.0
2016-01-08 07:00:00+00:00,0.0,0.0,0.0,-0.099020,0.732421,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,5,[],0.0
2016-01-11 07:00:00+00:00,0.0,0.0,0.0,-0.144346,0.684357,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,6,[],0.0
2016-01-12 07:00:00+00:00,0.0,0.0,0.0,-0.138112,0.653811,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,7,[],0.0
2016-01-13 07:00:00+00:00,0.0,0.0,0.0,-0.154148,0.605383,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,8,[],0.0
2016-01-14 07:00:00+00:00,0.0,0.0,0.0,-0.136541,0.606174,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,9,[],0.0
2016-01-15 07:00:00+00:00,0.0,0.0,0.0,-0.164105,0.577413,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,10,[],0.0


然后，我们可以定义一些函数来计算目标权重并根据我们的管道输出指定我们的多头和空头头寸的订单。让我们使用我们在入门教程中学到的一些基础知识来实现权重计算和下单。

最后，让我们把所有东西放在一起并记录一些变量。指定算法每周重新平衡。

In [8]:
%%zipline --start 2016-1-1 --end 2018-1-1
# 优化部分没完成
#from quantopian.algorithm import order_optimal_portfolio
from zipline.api import attach_pipeline, pipeline_output, schedule_function, date_rules, time_rules, record, order_percent
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage
from zipline.pipeline.builtin import QTradableStocksUS

#import quantopian.optimize as opt


def initialize(context):
    # Schedule our rebalance function to run at the start of
    # each week, when the market opens.
    schedule_function(my_rebalance, date_rules.week_start(),
                      time_rules.market_open())

    # Record variables at the end of each day.
    schedule_function(my_record_vars, date_rules.every_day(),
                      time_rules.market_close())

    # Create our pipeline and attach it to our algorithm.
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')


def make_pipeline():
    """
    Create our pipeline.
    """

    # Base universe set to the QTradableStocksUS.
    base_universe = QTradableStocksUS()

    # 10-day close price average.
    mean_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=10, mask=base_universe)

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30, mask=base_universe)

    percent_difference = (mean_10 - mean_30) / mean_30

    # Filter to select securities to short.
    shorts = percent_difference.top(75)

    # Filter to select securities to long.
    longs = percent_difference.bottom(75)

    # Filter for all securities that we want to trade.
    securities_to_trade = (shorts | longs)

    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        },
        screen=(securities_to_trade),
    )


def compute_target_weights(context, data):
    """
    Compute ordering weights.
    """

    # Initialize empty target weights dictionary.
    # This will map securities to their target weight.
    weights = {}

    # If there are securities in our longs and shorts lists,
    # compute even target weights for each security.
    if context.longs and context.shorts:
        long_weight = 0.5 / len(context.longs)
        short_weight = -0.5 / len(context.shorts)
    else:
        return weights

    # Exit positions in our portfolio if they are not
    # in our longs or shorts lists.
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(
                security):
            weights[security] = 0

    for security in context.longs:
        weights[security] = long_weight

    for security in context.shorts:
        weights[security] = short_weight

    return weights


def before_trading_start(context, data):
    """
    Get pipeline results.
    """

    # Gets our pipeline output every day.
    pipe_results = pipeline_output('my_pipeline')

    # Go long in securities for which the 'longs' value is True,
    # and check if they can be traded.
    context.longs = []
    for sec in pipe_results[pipe_results['longs']].index.tolist():
        if data.can_trade(sec):
            context.longs.append(sec)

    # Go short in securities for which the 'shorts' value is True,
    # and check if they can be traded.
    context.shorts = []
    for sec in pipe_results[pipe_results['shorts']].index.tolist():
        if data.can_trade(sec):
            context.shorts.append(sec)


def my_rebalance(context, data):
    """
    Rebalance weekly.
    """
    # Calculate target weights to rebalance
    target_weights = compute_target_weights(context, data)
    for s, w in target_weights.items():
        order_percent(s, w)
    # 不使用优化
    return
    # If we have target weights, rebalance our portfolio
    if target_weights:
        order_optimal_portfolio(
            objective=opt.TargetWeights(target_weights),
            constraints=[],
        )


def my_record_vars(context, data):
    """
    Record variables at the end of each day.
    """

    longs = shorts = 0
    for _, position in context.portfolio.positions.items():
        if position.amount > 0:
            longs += 1
        elif position.amount < 0:
            shorts += 1

    # Record our variables.
    record(
        leverage=context.account.leverage,
        long_count=longs,
        short_count=shorts)

[2018-04-27 21:42:12.503685] INFO: Performance: after split: asset: 财信发展(000838), amount: -36911, cost_basis: 15.27, last_sale_price: 55.68
[2018-04-27 21:42:12.504779] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:13.488470] INFO: Performance: after split: asset: 东方网力(300367), amount: -2755, cost_basis: 27.39, last_sale_price: 65.6
[2018-04-27 21:42:13.489502] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:14.410816] INFO: Performance: after split: asset: 宝德股份(300023), amount: -20370, cost_basis: 17.79, last_sale_price: 52.69
[2018-04-27 21:42:14.411953] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:15.708825] INFO: Performance: after split: asset: 易事特(300376), amount: -14048, cost_basis: 25.96, last_sale_price: 61.800000000000004
[2018-04-27 21:42:15.709921] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:16.802553] INFO: Performance: after split: asset: 融捷股份(002192), amount: -2777, cost_basis: 26.11, last_sale_price: 57.22
[2018-04-27 21:42:1

[2018-04-27 21:42:31.653929] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:32.167912] INFO: Performance: after split: asset: 横店东磁(002056), amount: -6096, cost_basis: 14.25, last_sale_price: 28.7
[2018-04-27 21:42:32.168766] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:32.169715] INFO: Performance: after split: asset: 杭锅股份(002534), amount: -18647, cost_basis: 11.43, last_sale_price: 16.26
[2018-04-27 21:42:32.170511] INFO: Performance: returning cash: 5.71
[2018-04-27 21:42:32.173480] INFO: Performance: after split: asset: 安琪酵母(600298), amount: -28453, cost_basis: 12.89, last_sale_price: 42.92
[2018-04-27 21:42:32.174363] INFO: Performance: returning cash: 6.45
[2018-04-27 21:42:32.175393] INFO: Performance: after split: asset: 赛轮金宇(601058), amount: -23485, cost_basis: 3.87, last_sale_price: 8.99
[2018-04-27 21:42:32.177614] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:32.688715] INFO: Performance: after split: asset: 天广中茂(002509), amount: 14784, cost_

[2018-04-27 21:42:38.209916] INFO: Performance: after split: asset: 天孚通信(300394), amount: 730, cost_basis: 27.91, last_sale_price: 86.0
[2018-04-27 21:42:38.210914] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:38.215267] INFO: Performance: after split: asset: 润达医疗(603108), amount: 3063, cost_basis: 27.68, last_sale_price: 79.66
[2018-04-27 21:42:38.216175] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:38.740679] INFO: Performance: after split: asset: 福晶科技(002222), amount: -5789, cost_basis: 11.36, last_sale_price: 16.87
[2018-04-27 21:42:38.742111] INFO: Performance: returning cash: 5.68
[2018-04-27 21:42:38.742831] INFO: Performance: after split: asset: 华策影视(300133), amount: -84, cost_basis: 15.32, last_sale_price: 23.98
[2018-04-27 21:42:38.744746] INFO: Performance: returning cash: 12.26
[2018-04-27 21:42:38.746418] INFO: Performance: after split: asset: 鲁亿通(300423), amount: 2314, cost_basis: 30.92, last_sale_price: 34.36
[2018-04-27 21:42:38.747420] INFO: Per

[2018-04-27 21:42:43.914971] INFO: Performance: returning cash: 15.11
[2018-04-27 21:42:43.916579] INFO: Performance: after split: asset: 乐凯新材(300446), amount: 3948, cost_basis: 39.71, last_sale_price: 79.08
[2018-04-27 21:42:43.917516] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:43.919597] INFO: Performance: after split: asset: 重庆港九(600279), amount: -14445, cost_basis: 8.31, last_sale_price: 10.41
[2018-04-27 21:42:43.921139] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:43.922350] INFO: Performance: after split: asset: 新通联(603022), amount: 6062, cost_basis: 21.52, last_sale_price: 49.94
[2018-04-27 21:42:43.923744] INFO: Performance: returning cash: 10.76
[2018-04-27 21:42:43.925401] INFO: Performance: after split: asset: 普莱柯(603566), amount: -3776, cost_basis: 25.44, last_sale_price: 44.160000000000004
[2018-04-27 21:42:43.927200] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:44.628481] INFO: Performance: after split: asset: 东方海洋(002086), amount: 2

[2018-04-27 21:42:50.774391] INFO: Performance: after split: asset: 深桑达Ａ(000032), amount: -4452, cost_basis: 15.04, last_sale_price: 17.46
[2018-04-27 21:42:50.775565] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:50.776806] INFO: Performance: after split: asset: 全信股份(300447), amount: 4548, cost_basis: 33.94, last_sale_price: 85.95
[2018-04-27 21:42:50.777878] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:50.778999] INFO: Performance: after split: asset: 浩云科技(300448), amount: 5438, cost_basis: 26.58, last_sale_price: 86.85000000000001
[2018-04-27 21:42:50.780083] INFO: Performance: returning cash: 2.54
[2018-04-27 21:42:50.783168] INFO: Performance: after split: asset: 恒瑞医药(600276), amount: -1926, cost_basis: 38.36, last_sale_price: 46.2
[2018-04-27 21:42:50.784285] INFO: Performance: returning cash: 0.0
[2018-04-27 21:42:50.785855] INFO: Performance: after split: asset: 农发种业(600313), amount: 19427, cost_basis: 4.24, last_sale_price: 14.030000000000001
[2018-04-27

...

[2018-04-27 21:46:50.279715] INFO: Performance: returning cash: 4.8
[2018-04-27 21:46:50.281791] INFO: Performance: after split: asset: 全志科技(300458), amount: 32322, cost_basis: 38.06, last_sale_price: 53.370000000000005
[2018-04-27 21:46:50.282421] INFO: Performance: returning cash: 15.57
[2018-04-27 21:46:51.301489] INFO: Performance: after split: asset: 特发信息(000070), amount: -7076, cost_basis: 15.16, last_sale_price: 22.18
[2018-04-27 21:46:51.302428] INFO: Performance: returning cash: 0.0
[2018-04-27 21:46:51.303462] INFO: Performance: after split: asset: 卫 士 通(002268), amount: 12825, cost_basis: 19.85, last_sale_price: 27.48
[2018-04-27 21:46:51.304140] INFO: Performance: returning cash: 11.91
[2018-04-27 21:46:51.305162] INFO: Performance: after split: asset: 盛通股份(002599), amount: 29898, cost_basis: 19.51, last_sale_price: 31.0
[2018-04-27 21:46:51.306402] INFO: Performance: returning cash: 0.0
[2018-04-27 21:46:51.308744] INFO: Performance: after split: asset: 哈尔斯(002615), amount

[2018-04-27 21:46:56.322046] INFO: Performance: after split: asset: 永东股份(002753), amount: -30827, cost_basis: 20.89, last_sale_price: 23.990000000000002
[2018-04-27 21:46:56.323169] INFO: Performance: returning cash: 10.45
[2018-04-27 21:46:56.324423] INFO: Performance: after split: asset: 东方国信(300166), amount: 12657, cost_basis: 16.06, last_sale_price: 20.48
[2018-04-27 21:46:56.325784] INFO: Performance: returning cash: 3.99
[2018-04-27 21:46:56.328805] INFO: Performance: after split: asset: 吉祥航空(603885), amount: 5416, cost_basis: 18.96, last_sale_price: 22.22
[2018-04-27 21:46:56.329586] INFO: Performance: returning cash: 11.38
[2018-04-27 21:46:57.453296] INFO: Performance: after split: asset: 合康新能(300048), amount: 24375, cost_basis: 7.43, last_sale_price: 7.94
[2018-04-27 21:46:57.454441] INFO: Performance: returning cash: 2.97
[2018-04-27 21:46:57.455413] INFO: Performance: after split: asset: 赢时胜(300377), amount: 25532, cost_basis: 15.7, last_sale_price: 31.63
[2018-04-27 21:46:

[2018-04-27 21:47:11.806928] INFO: Performance: returning cash: 7.4
[2018-04-27 21:47:11.810434] INFO: Performance: after split: asset: 光明地产(600708), amount: -16691, cost_basis: 7.83, last_sale_price: 8.83
[2018-04-27 21:47:11.811510] INFO: Performance: returning cash: 2.35
[2018-04-27 21:47:12.735505] INFO: Performance: after split: asset: 西藏天路(600326), amount: -70471, cost_basis: 9.15, last_sale_price: 11.040000000000001
[2018-04-27 21:47:12.736658] INFO: Performance: returning cash: 5.49
[2018-04-27 21:47:12.737747] INFO: Performance: after split: asset: 杭萧钢构(600477), amount: -68900, cost_basis: 9.39, last_sale_price: 15.26
[2018-04-27 21:47:12.738814] INFO: Performance: returning cash: 0.0
[2018-04-27 21:47:12.740251] INFO: Performance: after split: asset: 读者传媒(603999), amount: 26898, cost_basis: 16.3, last_sale_price: 22.32
[2018-04-27 21:47:12.741193] INFO: Performance: returning cash: 0.0
[2018-04-27 21:47:15.930726] INFO: Performance: after split: asset: 冰轮环境(000811), amount: -

[2018-04-27 21:48:04.940987] INFO: Performance: after split: asset: 北巴传媒(600386), amount: 10780, cost_basis: 8.05, last_sale_price: 13.35
[2018-04-27 21:48:04.941770] INFO: Performance: returning cash: 0.0
[2018-04-27 21:48:06.063941] INFO: Performance: after split: asset: 龙大肉食(002726), amount: 8294, cost_basis: 6.37, last_sale_price: 18.900000000000002
[2018-04-27 21:48:06.064905] INFO: Performance: returning cash: 1.91
[2018-04-27 21:48:19.060016] INFO: Performance: after split: asset: 三圣股份(002742), amount: -16786, cost_basis: 13.73, last_sale_price: 37.89
[2018-04-27 21:48:19.061060] INFO: Performance: returning cash: 0.0
[2018-04-27 21:48:26.799519] INFO: Performance: after split: asset: 新光圆成(002147), amount: -27158, cost_basis: 13.52, last_sale_price: 17.39
[2018-04-27 21:48:26.800625] INFO: Performance: returning cash: 13.52
[2018-04-27 21:48:26.801243] INFO: Performance: after split: asset: 北讯集团(002359), amount: 10378, cost_basis: 15.45, last_sale_price: 39.35
[2018-04-27 21:48:

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2016-01-04 05:35:00+00:00,NaN,0.000000,NaN,-0.070206,NaN,NaN,0.000000,1.000000e+07,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0,NaN,1.000000e+07,0.000000e+00,0.000000e+00,1,[],0.0
2016-01-05 07:00:00+00:00,0.006152,-0.000548,-0.132817,-0.067603,0.819490,-0.007507,-179413.517900,9.820586e+06,1.739328e+05,1.739328e+05,...,-4.686440e+06,-4.686440e+06,72,-11.224972,1.000000e+07,0.000000e+00,0.000000e+00,2,"[{'amount': 6144, 'dt': 2016-01-05 07:00:00+00...",0.0
2016-01-06 07:00:00+00:00,0.004678,-0.000634,-0.066177,-0.051246,0.745908,-0.003078,0.000000,9.820586e+06,1.730706e+05,1.730706e+05,...,-4.880259e+06,-4.880259e+06,72,-10.478794,9.820586e+06,1.739328e+05,1.739328e+05,3,[],0.0
2016-01-07 02:00:00+00:00,0.016306,-0.002842,-0.103588,-0.117026,0.738897,0.010057,0.000000,9.820586e+06,1.509978e+05,1.509978e+05,...,-4.413477e+06,-4.413477e+06,72,-9.909018,9.820586e+06,1.730706e+05,1.730706e+05,4,[],0.0
2016-01-08 07:00:00+00:00,0.016132,-0.002455,-0.059715,-0.099020,0.732421,0.012857,0.000000,9.820586e+06,1.548653e+05,1.548653e+05,...,-4.480854e+06,-4.480854e+06,72,-7.653794,9.820586e+06,1.509978e+05,1.509978e+05,5,[],0.0
2016-01-11 07:00:00+00:00,0.014629,-0.002574,-0.040727,-0.144346,0.684357,0.010764,0.000000,9.820586e+06,1.536733e+05,1.536733e+05,...,-4.116810e+06,-4.116810e+06,72,-7.316936,9.820586e+06,1.548653e+05,1.548653e+05,6,[],0.0
2016-01-12 07:00:00+00:00,0.072317,0.008813,0.532176,-0.138112,0.653811,0.041888,-313552.013419,9.507034e+06,5.810965e+05,5.810965e+05,...,-8.834417e+06,-8.834417e+06,111,23.263645,9.820586e+06,1.536733e+05,1.536733e+05,7,"[{'amount': -1781, 'dt': 2016-01-12 07:00:00+0...",0.0
2016-01-13 07:00:00+00:00,0.073841,0.015685,0.708720,-0.154148,0.605383,0.042668,0.000000,9.507034e+06,6.498164e+05,6.498164e+05,...,-8.362693e+06,-8.362693e+06,111,38.526432,9.507034e+06,5.810965e+05,5.810965e+05,8,[],0.0
2016-01-14 07:00:00+00:00,0.101999,0.032078,1.208478,-0.136541,0.606174,0.081409,59803.485840,9.566838e+06,7.539460e+05,7.539460e+05,...,-8.726829e+06,-8.726829e+06,111,73.774655,9.507034e+06,6.498164e+05,6.498164e+05,9,"[{'amount': -2916, 'dt': 2016-01-14 07:00:00+0...",0.0
2016-01-15 07:00:00+00:00,0.096183,0.035342,1.227639,-0.164105,0.577413,0.080211,0.000000,9.566838e+06,7.865868e+05,7.865868e+05,...,-8.265214e+06,-8.265214e+06,111,76.950753,9.566838e+06,7.539460e+05,7.539460e+05,10,[],0.0


**以上运行时间：8分19秒**

注意：当流水线在回测运行时，计算会分批执行以提高整体计算速度。由于计算是批量执行的，因此性能图表会显示为暂停。

祝贺您完成Pipeline教程！ 尝试在研究中设计一个管道，并在您自己的算法中使用它。